In [1]:
import numpy as np
import numpy_financial as npf
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf

import requests
import xlsxwriter
import math
from scipy import stats

# .map('{:,.2f}'.format) - - - for big numbers

In [2]:
stocks = pd.read_csv('/Users/jakemeissner/Documents/practice_projects_vscode/practice/algorithmic-trading-python-master/finished_files/sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN

In [3]:
#symbol = 'AMZN'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
#data = requests.get(api_url)
#data.status_code
data = requests.get(api_url).json()
data

NameError: name 'symbol' is not defined

In [4]:
data['year1ChangePercent']

NameError: name 'data' is not defined

In [5]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

for symbol_string in symbol_strings:
    print(symbol_string)

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,ADI,ADM,ADP,ADSK,AEE,AEP,AES,AFL,AIG,AIV,AIZ,AJG,AKAM,ALB,ALGN,ALK,ALL,ALLE,ALXN,AMAT,AMCR,AMD,AME,AMGN,AMP,AMT,AMZN,ANET,ANSS,ANTM,AON,AOS,APA,APD,APH,APTV,ARE,ATO,ATVI,AVB,AVGO,AVY,AWK,AXP,AZO,BA,BAC,BAX,BBY,BDX,BEN,BF.B,BIIB,BIO,BK,BKNG,BKR,BLK,BLL,BMY,BR,BRK.B,BSX,BWA,BXP,C,CAG,CAH,CARR,CAT,CB,CBOE,CBRE,CCI,CCL,CDNS,CDW,CE,CERN,CF,CFG,CHD,CHRW,CHTR,CI,CINF,CL,CLX,CMA,CMCSA
CME,CMG,CMI,CMS,CNC,CNP,COF,COG,COO,COP,COST,COTY,CPB,CPRT,CRM,CSCO,CSX,CTAS,CTL,CTSH,CTVA,CTXS,CVS,CVX,CXO,D,DAL,DD,DE,DFS,DG,DGX,DHI,DHR,DIS,DISCA,DISCK,DISH,DLR,DLTR,DOV,DOW,DPZ,DRE,DRI,DTE,DUK,DVA,DVN,DXC,DXCM,EA,EBAY,ECL,ED,EFX,EIX,EL,EMN,EMR,EOG,EQIX,EQR,ES,ESS,ETFC,ETN,ETR,EVRG,EW,EXC,EXPD,EXPE,EXR,F,FANG,FAST,FB,FBHS,FCX,FDX,FE,FFIV,FIS,FISV,FITB,FLIR,FLS,FLT,FMC,FOX,FOXA,FRC,FRT,FTI,FTNT,FTV,GD,GE,GILD
GIS,GL,GLW,GM,GOOG,GOOGL,GPC,GPN,GPS,GRMN,GS,GWW,HAL,HAS,HBAN,HBI,HCA,HD,HES,HFC,HIG,HII,HLT,HOLX,HON,HPE,HPQ,HRB,HRL,HSIC,HST,HSY,HUM,HWM,IBM,ICE,IDXX,IEX,IFF,ILM

In [6]:
final_df = pd.DataFrame(columns=my_columns)

for symbol_string in symbol_strings[:1]:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=price,stats,quote,news,chart&token={IEX_CLOUD_API_TOKEN}'
    #data = requests.get(batch_api_call_url)
    #print(data.status_code)
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_df = final_df.append(pd.Series(
            [
                symbol,
                data[symbol]['price'],
                data[symbol]['stats']['year1ChangePercent'],
                'N/A'
            ],
             index = my_columns),
              ignore_index=True)

final_df

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,150.593,0.156208,N/A
1,AAL,18.610,0.275000,N/A
2,AAP,238.030,0.388300,N/A
3,AAPL,183.410,0.369884,N/A
4,ABBV,139.920,0.324640,N/A
...,...,...,...,...
95,CINF,119.170,0.388135,N/A
96,CL,85.502,0.036331,N/A
97,CLX,189.510,-0.050312,N/A
98,CMA,98.620,0.635375,N/A


In [7]:
final_df = final_df.sort_values('One-Year Price Return', ascending=False)
final_df = final_df[:50]
final_df

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
37,ANET,135.130,0.775652,N/A
82,CBRE,109.590,0.697968,N/A
42,APA,34.157,0.676654,N/A
89,CF,69.230,0.673095,N/A
34,AMP,327.890,0.659372,N/A
98,CMA,98.620,0.635375,N/A
49,AVB,250.950,0.632461,N/A
54,AZO,2125.990,0.609104,N/A
29,AMAT,161.150,0.572350,N/A
18,AIG,60.890,0.572288,N/A


In [8]:
final_df = final_df.reset_index()

In [9]:
final_df

,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,37,ANET,135.130,0.775652,N/A
1,82,CBRE,109.590,0.697968,N/A
2,42,APA,34.157,0.676654,N/A
3,89,CF,69.230,0.673095,N/A
4,34,AMP,327.890,0.659372,N/A
5,98,CMA,98.620,0.635375,N/A
6,49,AVB,250.950,0.632461,N/A
7,54,AZO,2125.990,0.609104,N/A
8,29,AMAT,161.150,0.572350,N/A
9,18,AIG,60.890,0.572288,N/A


In [10]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input('Enter the size of your portfolio: ')

    try:
        float(portfolio_size)
    except ValueError: 
        print('This is not a number. Please try again.')
        portfolio_size = input('Enter the size of your portfolio: ')

portfolio_input()
print(portfolio_size)

25000


In [11]:
position_size = float(portfolio_size) / len(final_df.index)

for i in range(0, len(final_df)):
    #final_df.loc[i, 'Number of Shares to Buy'] = 0
    final_df.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/final_df.loc[i, 'Price'])
final_df

,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,37,ANET,135.130,0.775652,3
1,82,CBRE,109.590,0.697968,4
2,42,APA,34.157,0.676654,14
3,89,CF,69.230,0.673095,7
4,34,AMP,327.890,0.659372,1
5,98,CMA,98.620,0.635375,5
6,49,AVB,250.950,0.632461,1
7,54,AZO,2125.990,0.609104,0
8,29,AMAT,161.150,0.572350,3
9,18,AIG,60.890,0.572288,8


In [12]:
for symbol_string in symbol_strings:
    #print(symbol_string)
    for symbol in symbol_string.split(','):
        print(symbol)

A
AAL
AAP
AAPL
ABBV
ABC
ABMD
ABT
ACN
ADBE
ADI
ADM
ADP
ADSK
AEE
AEP
AES
AFL
AIG
AIV
AIZ
AJG
AKAM
ALB
ALGN
ALK
ALL
ALLE
ALXN
AMAT
AMCR
AMD
AME
AMGN
AMP
AMT
AMZN
ANET
ANSS
ANTM
AON
AOS
APA
APD
APH
APTV
ARE
ATO
ATVI
AVB
AVGO
AVY
AWK
AXP
AZO
BA
BAC
BAX
BBY
BDX
BEN
BF.B
BIIB
BIO
BK
BKNG
BKR
BLK
BLL
BMY
BR
BRK.B
BSX
BWA
BXP
C
CAG
CAH
CARR
CAT
CB
CBOE
CBRE
CCI
CCL
CDNS
CDW
CE
CERN
CF
CFG
CHD
CHRW
CHTR
CI
CINF
CL
CLX
CMA
CMCSA
CME
CMG
CMI
CMS
CNC
CNP
COF
COG
COO
COP
COST
COTY
CPB
CPRT
CRM
CSCO
CSX
CTAS
CTL
CTSH
CTVA
CTXS
CVS
CVX
CXO
D
DAL
DD
DE
DFS
DG
DGX
DHI
DHR
DIS
DISCA
DISCK
DISH
DLR
DLTR
DOV
DOW
DPZ
DRE
DRI
DTE
DUK
DVA
DVN
DXC
DXCM
EA
EBAY
ECL
ED
EFX
EIX
EL
EMN
EMR
EOG
EQIX
EQR
ES
ESS
ETFC
ETN
ETR
EVRG
EW
EXC
EXPD
EXPE
EXR
F
FANG
FAST
FB
FBHS
FCX
FDX
FE
FFIV
FIS
FISV
FITB
FLIR
FLS
FLT
FMC
FOX
FOXA
FRC
FRT
FTI
FTNT
FTV
GD
GE
GILD
GIS
GL
GLW
GM
GOOG
GOOGL
GPC
GPN
GPS
GRMN
GS
GWW
HAL
HAS
HBAN
HBI
HCA
HD
HES
HFC
HIG
HII
HLT
HOLX
HON
HPE
HPQ
HRB
HRL
HSIC
HST
HSY
HUM
HWM
IBM
ICE
IDXX
IEX
IFF
ILM

In [25]:
hqm_columns = [
'Ticker', 
'Price', 
'Number of Shares to Buy', 
'One-Year Price Return', 
'One-Year Return Percentile',
'Six-Month Price Return',
'Six-Month Return Percentile',
'Three-Month Price Return',
'Three-Month Return Percentile',
'One-Month Price Return',
'One-Month Return Percentile',
'HQM Score'
]


hqm_df = pd.DataFrame(columns=hqm_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=price,stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        hqm_df = hqm_df.append(pd.Series([symbol, 
            data[symbol]['quote']['latestPrice'],
            'N/A',
            data[symbol]['stats']['year1ChangePercent'],
            'N/A',
            data[symbol]['stats']['month6ChangePercent'],
            'N/A',
            data[symbol]['stats']['month3ChangePercent'],
            'N/A',
            data[symbol]['stats']['month1ChangePercent'],
            'N/A', 
            'N/A'
            ],
        index = hqm_columns), 
        ignore_index = True)
hqm_df

KeyError: 'WLTW'

In [26]:
hqm_df

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,152.876,N/A,0.156447,N/A,-0.020125,N/A,-0.021352,N/A,-0.060374,N/A,N/A
1,AAL,18.950,N/A,0.274,N/A,-0.091075,N/A,-0.056297,N/A,0.058099,N/A,N/A
2,AAP,239.120,N/A,0.380689,N/A,0.097467,N/A,0.099198,N/A,-0.030824,N/A,N/A
3,AAPL,182.567,N/A,0.367342,N/A,0.210825,N/A,0.231246,N/A,-0.025252,N/A,N/A
4,ABBV,137.070,N/A,0.322734,N/A,0.210036,N/A,0.257845,N/A,0.093619,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
479,WDC,66.450,N/A,0.294438,N/A,-0.060519,N/A,0.180975,N/A,0.14463,N/A,N/A
480,WEC,96.320,N/A,0.134841,N/A,0.05433,N/A,0.104217,N/A,0.020498,N/A,N/A
481,WELL,90.460,N/A,0.470594,N/A,0.013647,N/A,0.051711,N/A,0.044524,N/A,N/A
482,WFC,58.540,N/A,0.721362,N/A,0.289058,N/A,0.197281,N/A,0.119392,N/A,N/A
